In [9]:
import requests, pandas as pd, numpy as np
import time
from datetime import datetime
from bs4 import BeautifulSoup
from webdriver_manager.firefox import GeckoDriverManager
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import statistics

In [10]:
driver = webdriver.Firefox(executable_path=GeckoDriverManager().install())



====== WebDriver manager ======
Current firefox version is 100.0
Get LATEST geckodriver version for 100.0 firefox
Driver [C:\Users\spens\.wdm\drivers\geckodriver\win64\v0.31.0\geckodriver.exe] found in cache


In [11]:
def func(year, playtype, playoffs=False):
    url = 'https://www.nba.com/stats/players/{}/'
    url = url.format(playtype)
    #print(url)
    print(playtype)
    driver.get(url)
    time.sleep(2)
    
    season = Select(driver.find_element_by_xpath('/html/body/main/div/div/div[2]/div/div/div[1]/div[1]/div/div/label/select'))
    season.select_by_visible_text(year)
    time.sleep(1.5)
    print(year)
    
    season_type = Select(driver.find_element_by_xpath('/html/body/main/div/div/div[2]/div/div/div[1]/div[2]/div/div/label/select'))
    if not playoffs:
        season_type.select_by_visible_text('Regular Season')
    else:
        season_type.select_by_visible_text('Playoffs')
    
    try:
        l = driver.find_element_by_xpath('//*[@id="onetrust-close-btn-container"]')
        l.click()
        time.sleep(1.5)
    except:
        print("Skipped")
        time.sleep(1.5)
    all_pages = Select(driver.find_element_by_xpath('/html/body/main/div/div/div[2]/div/div/nba-stat-table/div[1]/div/div/select'))
    all_pages.select_by_visible_text('All')

    all_rows = driver.find_elements_by_xpath('/html/body/main/div/div/div[2]/div/div/nba-stat-table/div[2]/div[1]/table/tbody/tr')
    
    cols = []
    for i in range(1, 17):
            temp_url = '/html/body/main/div/div/div[2]/div/div/nba-stat-table/div[2]/div[1]/table/thead/tr/th[{}]'
            temp_url = temp_url.format(i)
            #print(temp_url)
            headers = driver.find_elements_by_xpath(temp_url)
            #print(headers[0].text)
            cols.append(headers[0].text)
    cols.append("PERCENTILE")
    
    
    data_lst = []

    size = len(all_rows) + 1
    for i in range(1, size):
        temp = []
        for j in range(1,18):
            url = '/html/body/main/div/div/div[2]/div/div/nba-stat-table/div[2]/div[1]/table/tbody/tr[{}]/td[{}]'
            data_url = url.format(i, j)
            data = driver.find_elements_by_xpath(data_url)
            data = data[0].text
            temp.append(data)
        data_lst.append(temp)
        
    
    data_lst = [[x.replace("%", "") for x in lst] for lst in data_lst]
    #print(data_lst)
    data_lst = [[float(x) if ('Sr.' not in x and 'Jr.' not in x and x.count('.') == 1) or x.isnumeric() else x for x in lst] for lst in data_lst]
    #print(data_lst)
    cols = [x.replace('\n', ' ') for x in cols]
    #print(cols)
    df = pd.DataFrame(columns=cols)
    #print(df)
    for lst in data_lst:
        df.loc[-1] = lst
        df.index = df.index + 1
        df = df.sort_index()
    
    df = df.sort_values(by="PTS", ascending=False)
    df.reset_index(drop=True, inplace=True)
    print("df")
    return df

In [12]:
years = ['2015-16', '2016-17', '2017-18', '2018-19', '2019-20', '2020-21', '2021-22']
cat_list = ['transition', 'isolation', 'ball-handler', 'roll-man', 'playtype-post-up', 'spot-up', 'hand-off','cut','off-screen']
trans_df = None
iso_df = None
pr_ball_df = None
pr_roll_df = None
post_df = None
spot_df = None
hand_df = None
cut_df = None
screen_df = None
putback_df = None
all_type_lst = []
for playtype in cat_list:
    df_lst = []
    for year in years:
        temp_df = func(year, playtype, playoffs=True)
        temp_df['Year'] = [year for i in range(len(temp_df))]
        temp_df['Playtype'] = [playtype for i in range(len(temp_df))]
        df_lst.append(temp_df)
    ptype_df = pd.concat(df_lst)
    all_type_lst.append(ptype_df)

transition
2015-16
df
transition
2016-17
Skipped
df
transition
2017-18
Skipped
df
transition
2018-19
Skipped
df
transition
2019-20
Skipped
df
transition
2020-21
Skipped
df
transition
2021-22
Skipped
df
isolation
2015-16
Skipped
df
isolation
2016-17
Skipped
df
isolation
2017-18
Skipped
df
isolation
2018-19
Skipped
df
isolation
2019-20
Skipped
df
isolation
2020-21
Skipped
df
isolation
2021-22
Skipped
df
ball-handler
2015-16
Skipped
df
ball-handler
2016-17
Skipped
df
ball-handler
2017-18
Skipped
df
ball-handler
2018-19
Skipped
df
ball-handler
2019-20
Skipped
df
ball-handler
2020-21
Skipped
df
ball-handler
2021-22
Skipped
df
roll-man
2015-16
Skipped
df
roll-man
2016-17
Skipped
df
roll-man
2017-18
Skipped
df
roll-man
2018-19
Skipped
df
roll-man
2019-20
Skipped
df
roll-man
2020-21
Skipped
df
roll-man
2021-22
Skipped
df
playtype-post-up
2015-16
Skipped
df
playtype-post-up
2016-17
Skipped
df
playtype-post-up
2017-18
Skipped
df
playtype-post-up
2018-19
Skipped
df
playtype-post-up
2019-20
Skippe

In [13]:
trans_df = all_type_lst[0]
iso_df = all_type_lst[1]
pr_ball_df = all_type_lst[2]
pr_roll_df = all_type_lst[3]
post_df = all_type_lst[4]
spot_df = all_type_lst[5]
hand_df = all_type_lst[6]
cut_df = all_type_lst[7]
screen_df = all_type_lst[8]

In [14]:
trans_df.to_csv('transition_playoffs_2016-22.csv')

In [15]:
iso_df.to_csv('isolation_playoffs_2016-22.csv')

In [16]:
pr_ball_df.to_csv('pr_ballhandler_playoffs_2016-22.csv')

In [17]:
pr_roll_df.to_csv('pr_rollman_playoffs_2016-22.csv')

In [18]:
post_df.to_csv('postup_playoffs_2016-22.csv')

In [19]:
spot_df.to_csv('spotup_playoffs_2016-22.csv')

In [20]:
hand_df.to_csv('handoff_playoffs_2016-22.csv')

In [21]:
cut_df.to_csv('cut_playoffs_2016-22.csv')

In [22]:
screen_df.to_csv('offscreen_playoffs_2016-22.csv')

In [33]:
trans_df.sort_values(by="PTS", ascending=False)

,PLAYER,TEAM,GP,POSS,FREQ,PPP,PTS,FGM,FGA,FG%,EFG%,FT FREQ,TOV FREQ,SF FREQ,AND ONE FREQ,SCORE FREQ,PERCENTILE,Year,Playtype
0,Giannis Antetokounmpo,MIL,11.0,8.5,25.5,1.18,10.0,3.1,5.4,57.6,60.2,35.5,9.7,36.6,9.7,60.2,62.8,2021-22,transition
0,Giannis Antetokounmpo,MIL,15.0,6.9,27.0,1.18,8.2,2.7,4.3,61.5,64.6,33.7,11.5,34.0,8.0,62.0,62.7,2018-19,transition
0,Russell Westbrook,OKC,6.0,7.3,21.0,1.11,8.2,2.7,5.0,53.0,60.0,18.2,15.9,15.9,2.3,52.3,54.1,2017-18,transition
1,John Wall,WAS,13.0,7.0,23.6,1.17,8.0,2.7,4.5,59.3,63.6,23.9,17.0,20.5,8.0,55.7,66.7,2016-17,transition
0,LeBron James,CLE,18.0,7.0,24.0,1.14,8.0,2.9,4.9,60.2,61.9,23.0,13.5,21.4,6.3,57.9,60.0,2016-17,transition
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,Cole Aldrich,LAC,6.0,0.2,5.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,2015-16,transition
176,Spencer Hawes,CHA,5.0,0.2,5.0,0.00,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2015-16,transition
175,Dorell Wright,MIA,5.0,0.2,8.3,0.00,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2015-16,transition
174,Jared Sullinger,BOS,6.0,0.2,2.0,0.00,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2015-16,transition


In [34]:
iso_df.sort_values(by="PTS", ascending=False)

,PLAYER,TEAM,GP,POSS,FREQ,PPP,PTS,FGM,FGA,FG%,EFG%,FT FREQ,TOV FREQ,SF FREQ,AND ONE FREQ,SCORE FREQ,PERCENTILE,Year,Playtype
0,James Harden,HOU,11.0,15.0,46.6,1.04,15.6,4.7,12.1,39.1,49.6,12.7,7.3,12.1,0.6,43.6,76.3,2018-19,isolation
0,Jayson Tatum,BOS,5.0,11.4,39.0,1.21,13.8,3.6,8.2,43.9,51.2,26.3,3.5,26.3,1.8,54.4,94.6,2020-21,isolation
0,James Harden,HOU,17.0,12.9,43.8,1.01,13.0,4.0,10.3,38.9,46.6,15.0,9.1,14.5,3.6,42.3,71.4,2017-18,isolation
1,Damian Lillard,POR,6.0,7.2,25.9,1.65,11.8,3.3,5.5,60.6,75.8,25.6,2.3,16.3,4.7,67.4,100.0,2020-21,isolation
0,James Harden,HOU,12.0,10.3,38.4,1.08,11.2,3.5,7.9,44.2,53.7,16.1,9.7,15.3,2.4,47.6,73.8,2019-20,isolation
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152,Ian Clark,GSW,16.0,0.4,9.1,0.00,0.0,0.0,0.3,0.0,0.0,0.0,33.3,0.0,0.0,0.0,0.0,2015-16,isolation
122,Montrezl Harrell,LAC,6.0,1.0,6.1,0.00,0.0,0.0,0.2,0.0,0.0,20.0,60.0,20.0,0.0,0.0,0.0,2018-19,isolation
121,Luke Kennard,DET,4.0,0.0,1.9,0.00,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,2018-19,isolation
120,Semi Ojeleye,BOS,6.0,0.0,9.1,0.00,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2018-19,isolation


In [35]:
pr_ball_df.sort_values(by="PTS", ascending=False)

,PLAYER,TEAM,GP,POSS,FREQ,PPP,PTS,FGM,FGA,FG%,EFG%,FT FREQ,TOV FREQ,SF FREQ,AND ONE FREQ,SCORE FREQ,PERCENTILE,Year,Playtype
0,Ja Morant,MEM,5.0,18.0,60.4,1.13,20.4,7.6,14.8,51.4,57.4,13.3,5.6,10.0,1.1,53.3,88.9,2020-21,ball-handler
0,Donovan Mitchell,UTA,7.0,15.6,51.7,1.19,19.0,6.6,12.0,54.8,63.0,12.8,11.9,11.0,1.8,53.2,88.5,2019-20,ball-handler
1,Trae Young,ATL,16.0,17.6,59.1,0.98,17.2,5.9,13.8,43.2,50.2,10.3,12.1,9.6,0.7,43.3,55.6,2020-21,ball-handler
2,Donovan Mitchell,UTA,10.0,14.6,49.7,1.16,17.0,5.6,11.3,49.6,60.2,15.1,11.0,13.0,3.4,50.0,92.1,2020-21,ball-handler
1,Luka Doncic,DAL,6.0,16.3,52.7,0.95,16.0,5.3,10.0,53.3,58.0,19.4,21.4,14.3,4.1,44.9,54.1,2019-20,ball-handler
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,Norris Cole,OKC,4.0,1.3,50.0,0.40,0.0,0.8,0.8,33.3,33.3,0.0,40.0,0.0,0.0,20.0,0.0,2016-17,ball-handler
127,Bojan Bogdanovic,WAS,13.0,0.5,6.5,0.29,0.0,0.3,0.3,0.0,0.0,28.6,14.3,14.3,0.0,28.6,0.0,2016-17,ball-handler
128,Andre Iguodala,GSW,16.0,0.5,6.2,0.25,0.0,0.4,0.4,16.7,16.7,0.0,25.0,0.0,0.0,12.5,0.0,2016-17,ball-handler
129,JR Smith,CLE,18.0,0.2,3.4,1.75,0.0,0.1,0.1,100.0,150.0,25.0,25.0,25.0,0.0,75.0,0.0,2016-17,ball-handler


In [36]:
pr_roll_df.sort_values(by="PTS", ascending=False)

,PLAYER,TEAM,GP,POSS,FREQ,PPP,PTS,FGM,FGA,FG%,EFG%,FT FREQ,TOV FREQ,SF FREQ,AND ONE FREQ,SCORE FREQ,PERCENTILE,Year,Playtype
0,Nikola Vucevic,ORL,5.0,8.2,32.0,1.56,12.8,5.0,8.0,62.5,76.3,4.9,0.0,4.9,2.4,63.0,100.0,2019-20,roll-man
0,Nikola Vucevic,CHI,5.0,8.2,40.6,1.05,8.6,3.6,8.0,45.0,53.8,0.0,2.4,0.0,0.0,43.9,52.6,2021-22,roll-man
0,DeAndre Jordan,LAC,7.0,5.0,35.6,1.54,8.1,3.7,4.0,93.0,93.0,18.9,8.1,18.9,2.7,83.8,96.9,2016-17,roll-man
0,Anthony Davis,NOP,9.0,5.3,18.8,1.48,7.9,3.3,4.7,71.4,72.6,14.6,2.1,14.6,4.2,72.9,87.1,2017-18,roll-man
1,Joel Embiid,PHI,4.0,7.3,25.0,1.07,7.8,2.3,5.0,45.0,47.5,27.6,3.4,20.7,0.0,55.0,40.7,2019-20,roll-man
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,Immanuel Quickley,NYK,5.0,0.0,2.4,0.00,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2020-21,roll-man
131,Trey Burke,DAL,2.0,0.0,14.3,0.00,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2020-21,roll-man
132,Josh Richardson,DAL,7.0,0.0,2.7,0.00,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2020-21,roll-man
133,Grayson Allen,MEM,5.0,0.0,2.9,0.00,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2020-21,roll-man


In [37]:
post_df.sort_values(by="PTS", ascending=False)

,PLAYER,TEAM,GP,POSS,FREQ,PPP,PTS,FGM,FGA,FG%,EFG%,FT FREQ,TOV FREQ,SF FREQ,AND ONE FREQ,SCORE FREQ,PERCENTILE,Year,Playtype
0,LaMarcus Aldridge,SAS,5.0,11.4,53.8,1.04,11.8,3.6,8.2,43.9,43.9,22.8,8.8,21.1,3.5,50.9,62.5,2017-18,playtype-post-up
0,Joel Embiid,PHI,4.0,9.8,33.6,1.15,11.3,3.0,6.0,53.8,53.8,28.2,10.3,20.5,5.1,59.0,92.9,2019-20,playtype-post-up
0,Joel Embiid,PHI,11.0,8.5,32.6,1.13,9.6,3.7,6.3,59.4,59.4,14.9,13.8,13.8,2.1,56.4,85.2,2020-21,playtype-post-up
0,Nikola Jokic,DEN,5.0,9.6,32.7,0.96,9.2,3.8,6.8,55.9,55.9,12.5,16.7,6.3,0.0,50.0,57.1,2021-22,playtype-post-up
1,Joel Embiid,PHI,8.0,10.0,41.0,0.81,8.1,2.6,6.8,38.9,38.9,21.3,15.0,16.3,3.8,43.8,37.5,2017-18,playtype-post-up
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,Maurice Harkless,POR,11.0,1.0,8.3,0.18,0.0,0.5,0.5,16.7,16.7,27.3,9.0,27.0,0.0,9.0,0.0,2015-16,playtype-post-up
92,Mike Muscala,ATL,6.0,0.2,4.3,2.00,0.0,0.2,0.2,100.0,100.0,0.0,0.0,0.0,0.0,100.0,0.0,2016-17,playtype-post-up
99,Mason Plumlee,POR,11.0,0.5,4.5,0.40,0.0,0.4,0.4,25.0,25.0,0.0,20.0,0.0,0.0,20.0,0.0,2015-16,playtype-post-up
98,Ian Mahinmi,IND,7.0,0.6,5.9,0.50,0.0,0.4,0.4,33.3,33.3,0.0,25.0,0.0,0.0,25.0,0.0,2015-16,playtype-post-up


In [38]:
spot_df.sort_values(by="PTS", ascending=False)

,PLAYER,TEAM,GP,POSS,FREQ,PPP,PTS,FGM,FGA,FG%,EFG%,FT FREQ,TOV FREQ,SF FREQ,AND ONE FREQ,SCORE FREQ,PERCENTILE,Year,Playtype
0,Al-Farouq Aminu,POR,4.0,9.0,59.3,1.00,12.0,4.3,8.3,51.5,69.7,0.0,5.7,0.0,0.0,48.6,81.5,2017-18,spot-up
0,Timothe Luwawu-Cabarrot,BKN,4.0,9.5,52.8,1.05,10.0,3.3,8.8,37.1,52.9,2.6,5.3,2.6,0.0,36.8,45.6,2019-20,spot-up
2,Kristaps Porzingis,DAL,3.0,5.3,30.8,1.75,9.3,2.7,4.7,57.1,75.0,18.8,0.0,18.8,6.3,62.5,97.4,2019-20,spot-up
1,Tyler Johnson,BKN,4.0,9.0,72.0,1.03,9.3,3.3,8.5,38.2,51.5,2.8,2.8,2.8,0.0,38.9,42.1,2019-20,spot-up
3,Joe Harris,BKN,2.0,6.5,46.4,1.39,9.0,3.0,6.0,50.0,70.8,7.7,0.0,7.7,0.0,53.8,86.8,2019-20,spot-up
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164,Trey Burke,WAS,3.0,0.7,50.0,0.00,0.0,0.0,0.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2016-17,spot-up
165,Noah Vonleh,POR,4.0,1.3,18.5,0.00,0.0,0.0,1.0,0.0,0.0,0.0,20.0,0.0,0.0,0.0,0.0,2016-17,spot-up
166,Mirza Teletovic,MIL,3.0,1.7,71.4,0.00,0.0,0.0,1.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2016-17,spot-up
167,Meyers Leonard,POR,3.0,1.3,44.4,0.00,0.0,0.0,1.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2016-17,spot-up


In [39]:
hand_df.sort_values(by="PTS", ascending=False)

,PLAYER,TEAM,GP,POSS,FREQ,PPP,PTS,FGM,FGA,FG%,EFG%,FT FREQ,TOV FREQ,SF FREQ,AND ONE FREQ,SCORE FREQ,PERCENTILE,Year,Playtype
0,Michael Porter Jr.,DEN,10.0,3.1,19.5,1.32,4.1,1.5,3.0,48.3,63.8,6.5,3.2,6.5,3.0,48.4,79.2,2020-21,hand-off
0,Jamal Murray,DEN,19.0,3.7,15.7,1.09,4.1,1.6,2.8,56.6,62.3,9.9,16.9,8.5,1.4,49.3,82.8,2019-20,hand-off
1,Duncan Robinson,MIA,21.0,3.3,33.8,1.06,3.5,1.0,2.9,36.1,51.6,8.7,7.2,8.7,4.3,36.2,69.0,2019-20,hand-off
0,Stephen Curry,GSW,11.0,3.0,12.2,1.12,3.4,1.1,2.3,48.0,64.0,12.1,12.1,9.1,0.0,45.5,61.1,2021-22,hand-off
2,Tyler Herro,MIA,21.0,2.9,17.4,1.03,3.0,1.1,2.5,43.4,53.8,5.0,6.7,3.3,0.0,43.3,65.5,2019-20,hand-off
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,Nikola Jokic,DEN,19.0,0.1,0.5,0.00,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2019-20,hand-off
93,Luguentz Dort,OKC,6.0,0.3,2.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,2019-20,hand-off
92,Quinn Cook,LAL,5.0,0.4,18.2,0.00,0.0,0.0,0.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2019-20,hand-off
91,JR Smith,LAL,7.0,0.1,3.0,0.00,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2019-20,hand-off


In [40]:
cut_df.sort_values(by="PTS", ascending=False)

,PLAYER,TEAM,GP,POSS,FREQ,PPP,PTS,FGM,FGA,FG%,EFG%,FT FREQ,TOV FREQ,SF FREQ,AND ONE FREQ,SCORE FREQ,PERCENTILE,Year,Playtype
0,Anthony Davis,NOP,9.0,4.0,14.1,1.56,6.2,2.8,3.6,78.1,78.1,11.1,6.0,8.3,5.6,75.0,90.0,2017-18,cut
0,Rudy Gobert,UTA,7.0,4.6,31.4,1.38,6.0,2.6,4.0,64.0,64.0,21.9,0.0,21.9,9.4,68.8,66.7,2019-20,cut
1,Clint Capela,HOU,17.0,3.5,32.4,1.53,5.4,2.6,3.3,80.4,80.4,3.3,5.0,3.3,1.7,76.7,87.5,2017-18,cut
0,Clint Capela,HOU,11.0,4.5,47.2,1.12,5.1,2.0,3.6,55.0,55.0,22.0,6.0,22.0,8.0,58.0,29.3,2016-17,cut
1,Greg Monroe,MIL,6.0,3.7,28.6,1.36,5.0,2.0,3.0,66.7,66.7,18.2,9.1,13.6,9.1,63.6,75.6,2016-17,cut
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,Tyler Johnson,BKN,4.0,0.3,2.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2019-20,cut
128,Markelle Fultz,ORL,5.0,0.2,1.3,2.00,0.0,0.0,0.0,0.0,0.0,100.0,0.0,100.0,0.0,100.0,0.0,2019-20,cut
127,Bol Bol,DEN,4.0,0.3,6.7,2.00,0.0,0.0,0.0,0.0,0.0,100.0,0.0,100.0,0.0,100.0,0.0,2019-20,cut
126,Danuel House Jr.,HOU,9.0,0.9,8.1,0.38,0.0,0.1,0.7,17.0,17.0,12.5,12.5,0.0,0.0,25.0,0.0,2019-20,cut


In [31]:
screen_df

,PLAYER,TEAM,GP,POSS,FREQ,PPP,PTS,FGM,FGA,FG%,EFG%,FT FREQ,TOV FREQ,SF FREQ,AND ONE FREQ,SCORE FREQ,PERCENTILE,Year,Playtype
0,Klay Thompson,GSW,24.0,6.8,30.2,1.07,7.3,2.6,6.3,41.1,53.6,4.9,3.7,4.9,1.2,41.1,58.3,2015-16,off-screen
1,JJ Redick,LAC,6.0,5.3,37.6,1.03,5.5,2.3,4.8,48.3,56.9,3.1,9.4,3.1,3.1,43.8,50.0,2015-16,off-screen
2,Kevin Durant,OKC,18.0,4.7,16.1,1.08,5.1,2.1,4.1,50.0,56.1,4.7,8.2,4.7,0.0,48.2,62.5,2015-16,off-screen
3,Kyle Korver,ATL,10.0,2.5,29.4,1.76,4.4,1.4,2.4,58.3,85.4,4.0,4.0,4.0,0.0,60.0,100.0,2015-16,off-screen
4,Paul George,IND,7.0,4.6,18.2,0.97,4.4,1.1,3.3,34.8,43.5,18.8,9.4,15.6,0.0,43.8,41.7,2015-16,off-screen
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,Pascal Siakam,TOR,6.0,0.2,0.7,0.00,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2021-22,off-screen
100,Bruce Brown,BKN,4.0,0.3,2.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,2021-22,off-screen
101,Juan Toscano-Anderson,GSW,6.0,0.2,7.7,0.00,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,2021-22,off-screen
102,Goran Dragic,BKN,4.0,0.3,2.7,0.00,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,2021-22,off-screen
